In [7]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [8]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

In [9]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [10]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [11]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2021-09-16 14:34:38,212 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2021-09-16 14:34:38,213 - mmcv - INFO - load model from: torchvision://resnet50
2021-09-16 14:34:38,213 - mmcv - INFO - Use load_from_torchvision loader
2021-09-16 14:34:38,429 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2021-09-16 14:34:38,453 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2021-09-16 14:34:38,480 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2021-09-16 14:34:38,489 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'layer': 'Linear', 'override': [{'name': 'shared_fcs'}, {'name': '

2021-09-16 14:34:38,844 - mmcv - INFO - 
backbone.layer2.1.bn2.weight - torch.Size([128]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,845 - mmcv - INFO - 
backbone.layer2.1.bn2.bias - torch.Size([128]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,845 - mmcv - INFO - 
backbone.layer2.1.conv3.weight - torch.Size([512, 128, 1, 1]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,846 - mmcv - INFO - 
backbone.layer2.1.bn3.weight - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,847 - mmcv - INFO - 
backbone.layer2.1.bn3.bias - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,847 - mmcv - INFO - 
backbone.layer2.2.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,848 - mmcv - INFO - 
backbone.layer2.2.bn1.weight - torch.Size([128]): 
PretrainedInit: load f

2021-09-16 14:34:38,897 - mmcv - INFO - 
backbone.layer3.3.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,898 - mmcv - INFO - 
backbone.layer3.3.bn2.weight - torch.Size([256]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,899 - mmcv - INFO - 
backbone.layer3.3.bn2.bias - torch.Size([256]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,900 - mmcv - INFO - 
backbone.layer3.3.conv3.weight - torch.Size([1024, 256, 1, 1]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,901 - mmcv - INFO - 
backbone.layer3.3.bn3.weight - torch.Size([1024]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,902 - mmcv - INFO - 
backbone.layer3.3.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-16 14:34:38,902 - mmcv - INFO - 
backbone.layer3.4.conv1.weight - torch.Size([256, 1024, 1, 1]): 
Pretr

2021-09-16 14:34:38,941 - mmcv - INFO - 
neck.lateral_convs.1.conv.weight - torch.Size([256, 512, 1, 1]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
2021-09-16 14:34:38,942 - mmcv - INFO - 
neck.lateral_convs.1.conv.bias - torch.Size([256]): 
The value is the same before and after calling `init_weights` of FasterRCNN  
 
2021-09-16 14:34:38,942 - mmcv - INFO - 
neck.lateral_convs.2.conv.weight - torch.Size([256, 1024, 1, 1]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
2021-09-16 14:34:38,943 - mmcv - INFO - 
neck.lateral_convs.2.conv.bias - torch.Size([256]): 
The value is the same before and after calling `init_weights` of FasterRCNN  
 
2021-09-16 14:34:38,944 - mmcv - INFO - 
neck.lateral_convs.3.conv.weight - torch.Size([256, 2048, 1, 1]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
2021-09-16 14:34:38,944 - mmcv - INFO - 
neck.lateral_convs.3.conv.bias - torch.Size([256]): 
The value is the same before and after calling `init_weights` of FasterRCNN  
 

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2021-09-16 14:34:39,052 - mmdet - INFO - Start running, host: root@25a605f857fd, work_dir: /opt/ml/detection/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash
2021-09-16 14:34:39,053 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) CheckpointHook                     
(LOW         ) IterTimerHook                      
(V

2021-09-16 14:39:57,378 - mmdet - INFO - Epoch [2][100/1221]	lr: 2.000e-02, eta: 0:52:33, time: 0.238, data_time: 0.007, memory: 2449, loss_rpn_cls: 0.0690, loss_rpn_bbox: 0.0463, loss_cls: 0.3660, acc: 90.5518, loss_bbox: 0.2750, loss: 0.7564, grad_norm: 1.7947
2021-09-16 14:40:09,388 - mmdet - INFO - Epoch [2][150/1221]	lr: 2.000e-02, eta: 0:52:22, time: 0.240, data_time: 0.006, memory: 2449, loss_rpn_cls: 0.0708, loss_rpn_bbox: 0.0427, loss_cls: 0.3481, acc: 90.7354, loss_bbox: 0.2653, loss: 0.7270, grad_norm: 1.7982
2021-09-16 14:40:20,873 - mmdet - INFO - Epoch [2][200/1221]	lr: 2.000e-02, eta: 0:52:07, time: 0.230, data_time: 0.006, memory: 2449, loss_rpn_cls: 0.0753, loss_rpn_bbox: 0.0372, loss_cls: 0.3315, acc: 91.4971, loss_bbox: 0.2497, loss: 0.6937, grad_norm: 1.7816
2021-09-16 14:40:32,365 - mmdet - INFO - Epoch [2][250/1221]	lr: 2.000e-02, eta: 0:51:53, time: 0.230, data_time: 0.006, memory: 2449, loss_rpn_cls: 0.0712, loss_rpn_bbox: 0.0371, loss_cls: 0.3189, acc: 91.8311,